In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-04-06


Found rundate 2024-04-05 10:00:02 < 2024-04-06 at commit b4de6540; returning commit 2b620a9


'2024-04-06 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-04-06 10:00:03, NJSP was reporting 153 YTD deaths
Current YTD Deaths (2025-04-06 10:00:01-04:00): 118
Previous year YTD Deaths (adjusted; 2024-04-06 10:00:03): 151.82724371978912
Projected 2025 total: 622.0159710438508
-22.3% change


FAUQStats cache miss: 2024, Sat Apr 06 10:00:03 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,East Oak St,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,27,State Highway 27,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13099,15,Ocean,1530,Stafford Twsp,539,County 539 MP 11.5,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-04-02 09:01:00-04:00
13097,13,Monmouth,1319,Howell Twsp,33,State Highway 33,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-04-02 10:28:00-04:00
13100,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-04-03 14:15:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,5,0,3,0,8
Bergen,5,1,1,0,7
Burlington,4,2,3,0,9
Camden,9,0,3,0,12
Cape May,2,0,0,0,2
Cumberland,3,1,2,0,5
Essex,1,0,5,1,7
Gloucester,5,2,0,0,5
Hudson,0,0,2,0,2


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

26.1% through the year, 73.9% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,5,31,5
Bergen,driver,6,14,5
Burlington,driver,8,32,4
Camden,driver,5,19,9
Cape May,driver,1,8,2
...,...,...,...,...
Salem,crashes,2,14,2
Somerset,crashes,2,18,6
Sussex,crashes,1,12,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,47,1,31,1,16
Bergen,30,1,13,5,13
Burlington,41,1,25,4,11
Camden,39,0,26,2,11
Cape May,12,0,11,0,1
Cumberland,31,0,21,6,8
Essex,49,2,19,6,28
Gloucester,25,0,18,8,4
Hudson,15,1,6,0,10


In [9]:
projected.to_csv(PROJECTED_CSV)